<a href="https://colab.research.google.com/github/HajarHAMDOUCH01/Mod-leStatistique/blob/main/ModelStats_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.listdir()

['.config', 'sample_data']

In [2]:
from google.colab import files
uploaded = files.upload()

Saving xgb_train_preds.npy to xgb_train_preds.npy
Saving xgb_test_preds (2).npy to xgb_test_preds (2).npy


In [3]:
from google.colab import files
uploaded = files.upload()

Saving nn_train_preds.npy to nn_train_preds.npy
Saving nn_test_preds.npy to nn_test_preds.npy


In [10]:
!pip install ucimlrepo
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from ucimlrepo import fetch_ucirepo

covertype = fetch_ucirepo(id=31)
X = covertype.data.features
y = covertype.data.targets

y = to_categorical(y, num_classes=8)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

nn_train_preds = np.load("nn_train_preds.npy")
nn_test_preds = np.load("nn_test_preds.npy")

xgb_train_preds = np.load("xgb_train_preds.npy")
xgb_test_preds = np.load("xgb_test_preds.npy")

xgb_train_preds = xgb_train_preds.reshape(-1, 1)
xgb_test_preds = xgb_test_preds.reshape(-1, 1)

train_meta_features = np.hstack([nn_train_preds, xgb_train_preds])
test_meta_features = np.hstack([nn_test_preds, xgb_test_preds])

print("Shape of training meta-features:", train_meta_features.shape)
print("Shape of testing meta-features:", test_meta_features.shape)

log_reg_model = LogisticRegression(max_iter=500, random_state=42)
log_reg_model.fit(train_meta_features, y_train)

meta_test_preds = log_reg_model.predict(test_meta_features)

meta_accuracy = accuracy_score(y_test, meta_test_preds)
print("Final Logistic Regression Model Accuracy: {:.2f}%".format(meta_accuracy * 100))

print("\nClassification Report:\n")
print(classification_report(y_test, meta_test_preds, target_names=[f"Class {i}" for i in range(7)]))


Shape of training meta-features: (464809, 9)
Shape of testing meta-features: (116203, 9)
Final Logistic Regression Model Accuracy: 89.36%

Classification Report:

              precision    recall  f1-score   support

     Class 0       0.90      0.88      0.89     42557
     Class 1       0.90      0.92      0.91     56500
     Class 2       0.86      0.92      0.89      7121
     Class 3       0.84      0.68      0.76       526
     Class 4       0.74      0.62      0.68      1995
     Class 5       0.81      0.78      0.79      3489
     Class 6       0.92      0.89      0.91      4015

    accuracy                           0.89    116203
   macro avg       0.85      0.81      0.83    116203
weighted avg       0.89      0.89      0.89    116203

